In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
imdb_data=pd.read_csv('IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [4]:
imdb_data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
imdb_data.isna().sum()

review       0
sentiment    0
dtype: int64

In [6]:
imdb_data.duplicated().sum()

np.int64(418)

In [7]:
imdb_data.drop_duplicates(inplace=True)

In [8]:
imdb_data['sentiment'].value_counts()

sentiment
positive    24884
negative    24698
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X = imdb_data.iloc[:,0]
y = imdb_data.iloc[:,1]

In [10]:
X_test , X_train , y_test , y_train = train_test_split(X, y , test_size=0.2,random_state = 1)

In [11]:
print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)

(39665,)
(9917,)
(39665,)
(9917,)


In [12]:
nltk.download('stopwords')
tokenizer=ToktokTokenizer()
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [14]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [15]:
import nltk
from nltk.stem import PorterStemmer
import swifter

ps = PorterStemmer()

def simple_stemmer(text):
    return ' '.join(ps.stem(word) for word in text.split())

imdb_data['review'] = imdb_data['review'].swifter.apply(simple_stemmer)

Pandas Apply:   0%|          | 0/49582 [00:00<?, ?it/s]

In [16]:
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from multiprocessing.dummy import Pool as ThreadPool
from tqdm import tqdm

stopword_list = set(stopwords.words('english'))

def remove_stopwords(text, is_lower_case=False):
    tokens = word_tokenize(text)
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    return ' '.join(filtered_tokens)

def parallel_apply_with_threads(series, func, num_threads=8, chunksize=1000):
    chunks = np.array_split(series, max(1, len(series) // chunksize))
    results = []
    with ThreadPool(num_threads) as pool:
        for result in tqdm(pool.imap(lambda s: s.apply(func), chunks), total=len(chunks), desc="Processing"):
            results.append(result)
    return pd.concat(results)

imdb_data['review'] = parallel_apply_with_threads(imdb_data['review'], remove_stopwords, num_threads=8)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\monis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Processing: 100%|██████████████████████████████████████████████████████████████████████| 49/49 [01:09<00:00,  1.43s/it]


In [35]:
imdb_data.to_csv('processed_imdb.csv', index=False)

In [17]:
norm_train_reviews=  X_test
norm_train_reviews

17824    I got this movie from the library, and saw it ...
41082    This may have been made for the hell of it, bu...
35822    I saw this movie when I was a child. It blew m...
40       It had all the clichés of movies of this type ...
19775    Cult starts 20 years ago on the 'Quinling Moun...
                               ...                        
44035    It helps that the characters this show is base...
32696    Carlos Mencia continually, violently, hatefull...
5195     A lot of horror fans seem to love Scarecrows, ...
12195    In the first 20 minutes, every cliche possible...
33189    I thought I was wasting my precious 50 bucks g...
Name: review, Length: 39665, dtype: object

In [18]:
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[45005]

'read review watch thi piec cinemat garbag took least 2 page find somebodi els didnt think thi appallingli unfunni montag wasnt acm humour 70 inde ani era thi isnt least funni set sketch comedi ive ever seen itll till come along half skit alreadi done infinit better act monti python woodi allen wa say nice piec anim last 90 second highlight thi film would still get close sum mindless drivelridden thi wast 75 minut semin comedi onli world semin realli doe mean semen scatolog humour onli world scat actual fece precursor joke onli mean thi handbook comedi tit bum odd beaver niceif pubesc boy least one hand free havent found playboy exist give break becaus wa earli 70 way sketch comedi go back least ten year prior onli way could even forgiv thi film even made wa gunpoint retro hardli sketch clown subtli pervert children may cut edg circl could actual funni come realli quit sad kept go throughout entir 75 minut sheer belief may save genuin funni skit end gave film 1 becaus wa lower scoreand

In [19]:
from sklearn.model_selection import train_test_split
X1 = imdb_data.iloc[:,0]
y1 = imdb_data.iloc[:,1]

In [20]:
X_test , X_train , y_test , y_train = train_test_split(X1, y1 , test_size=0.2,random_state = 1)

In [21]:
norm_test_reviews=  X_test
norm_test_reviews.iloc[0]

'got thi movi librari saw lot actor like itjohn clees ian holm ralph richardson etc got watch expect clees larg role sinc first bill wa surpris find five minut screen time along everyon els like thi movi amazingli pointless charact nobodi plot nonexist end one worst end ever seen funni part stay away thi movi want prevent go huh lot dont want wast time ignor peopl say thi veri funni movieit isnt stay away costspleas'

In [22]:
norm_train_reviews=  X_train
norm_train_reviews.iloc[0]

'saw thi show 34 year ago wa dam funni first time saw wa play etvestonian televis start like bad show bad time hyde like cool guy like sing frank sinatra come stupid idea got glass h brown like fez group pervert know doe hi alon want get laid badli even hi boss one episodehi india michael stupidest guy whole group probabl stupidest town hi cop stupid rememb follow hyde say u call cop michael come say doe anyon know turn siren town playboy come jacki former girlfriend michael hyde girlfriend eric son grumpi war veteran son kitti housewif hi one big pussi love donna hi girlfriend plan theyr marriag donna one hot girl hmm forget ah hyde live basement'

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    min_df=1,              # keep tokens that appear in at least 1 document
    max_df=0.9,            # drop tokens appearing in >90% of documents (removes common words)
    binary=False,          # keep raw counts instead of binary presence
    ngram_range=(1, 3),    # unigrams, bigrams, trigrams
    max_features=50000     # cap the vocabulary size to save memory
)

cv_train_reviews = cv.fit_transform(norm_train_reviews)
cv_test_reviews = cv.transform(norm_test_reviews)

print('BOW_cv_train:', cv_train_reviews.shape)
print('BOW_cv_test:', cv_test_reviews.shape)

BOW_cv_train: (9917, 50000)
BOW_cv_test: (39665, 50000)


In [24]:
cv_train_reviews

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1324601 stored elements and shape (9917, 50000)>

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(
    min_df=1,              # keep tokens that appear in at least 1 document
    max_df=0.9,            # drop tokens appearing in >90% of documents
    use_idf=True,          # enable inverse document frequency weighting
    ngram_range=(1, 3),    # unigrams, bigrams, trigrams
    max_features=50000,    # cap vocabulary size for memory efficiency
    sublinear_tf=True      # optional: log scale term frequency
)

# Transform
tv_train_reviews = tv.fit_transform(norm_train_reviews)
tv_test_reviews = tv.transform(norm_test_reviews)

print('Tfidf_train:', tv_train_reviews.shape)
print('Tfidf_test:', tv_test_reviews.shape)

Tfidf_train: (9917, 50000)
Tfidf_test: (39665, 50000)


In [26]:
lb=LabelBinarizer()

sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(49582, 1)


In [27]:
X_train, X_test, y_train_text, y_test_text = train_test_split(imdb_data['review'],imdb_data['sentiment'],
    test_size=0.2,random_state=1)

lb = LabelBinarizer()
train_sentiments = lb.fit_transform(y_train_text)
test_sentiments = lb.transform(y_test_text)


In [28]:
print(train_sentiments.shape)
print(test_sentiments.shape)

(39665, 1)
(9917, 1)


In [29]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=1, n_jobs=-1)
# interchanged train/test somewhere by mistake
lr_bow=lr.fit(cv_test_reviews,train_sentiments)
print(lr_bow)

lr_tfidf=lr.fit(tv_test_reviews,train_sentiments)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, n_jobs=-1, random_state=1)
LogisticRegression(C=1, max_iter=500, n_jobs=-1, random_state=1)


In [30]:
lr_bow_predict=lr.predict(cv_train_reviews)
print(lr_bow_predict.shape)

lr_tfidf_predict=lr.predict(tv_train_reviews)
print(lr_tfidf_predict.shape)

(9917,)
(9917,)


In [31]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)    

lr_bow_score : 0.8724412624785721
lr_tfidf_score : 0.8993647272360593


In [63]:
lr_gcv = LogisticRegression(max_iter=300)

param_grid = {
    'penalty': ['elasticnet'],     
    'C': [0.1,1,10],
    'l1_ratio': [0,0.5,0.75,1],            
    'solver': ['saga']
}

In [64]:
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
results = []
pg = list(ParameterGrid(param_grid))

for params in tqdm(pg, desc="Grid Search Progress"):
    lr = LogisticRegression(**params)
    scores = cross_val_score(lr_gcv, cv_test_reviews, train_sentiments, cv=3, scoring='accuracy', n_jobs=-1 , verbose=3)
    results.append((params, np.mean(scores)))

# Get best params
best_params, best_score = max(results, key=lambda x: x[1])
print("\nBest params:", best_params)
print("Best score:", best_score)

# grid = GridSearchCV(estimator=lr_gcv ,param_grid=param_grid ,scoring='accuracy',  n_jobs=-1 , verbose=3 , cv= 3)

# grid_bow = grid.fit(cv_test_reviews,train_sentiments)
# print("Best params:", grid_bow.best_params_)
# print("Best score:", grid_bow.best_score_)
# print()
# grid_tfidf = grid.fit(tv_test_reviews,train_sentiments)
# print("Best params:", grid_tfidf.best_params_)
# print("Best score:", grid_tfidf.best_score_)

Grid Search Progress:   0%|                                                                     | 0/12 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   15.9s finished
Grid Search Progress:   8%|█████                                                        | 1/12 [00:16<02:59, 16.35s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   15.2s finished
Grid Search Progress:  17%|██████████▏                                                  | 2/12 [00:31<02:39, 15.93s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   15.4s finished
Grid Search Progress:  25%|███████████████▎                                             | 3/12 [00:47<02:22, 15.81s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Pa


Best params: {'C': 0.1, 'l1_ratio': 0, 'penalty': 'elasticnet', 'solver': 'saga'}
Best score: 0.8859447531277667


In [66]:
results_tfidf = []
pg = list(ParameterGrid(param_grid))

for params in tqdm(pg, desc="Grid Search Progress"):
    lr = LogisticRegression(**params)
    scores = cross_val_score(lr_gcv, tv_test_reviews,train_sentiments, cv=3, scoring='accuracy', n_jobs=-1 , verbose=3)
    results.append((params, np.mean(scores)))

# Get best params
best_params_tfidf, best_score_tfidf = max(results, key=lambda x: x[1])
print("\nBest params:", best_params_tfidf)
print("Best score:", best_score_tfidf)

Grid Search Progress:   0%|                                                                     | 0/12 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.6s finished
Grid Search Progress:   8%|█████                                                        | 1/12 [00:02<00:31,  2.85s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    8.8s finished
Grid Search Progress:  17%|██████████▏                                                  | 2/12 [00:12<01:06,  6.61s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    9.2s finished
Grid Search Progress:  25%|███████████████▎                                             | 3/12 [00:21<01:12,  8.00s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Pa


Best params: {'C': 0.1, 'l1_ratio': 0, 'penalty': 'elasticnet', 'solver': 'saga'}
Best score: 0.8952729746907592


In [68]:
# Retrain the best model on the full training data
best_model = LogisticRegression(**best_params_tfidf)
best_model.fit(tv_test_reviews,train_sentiments)

,penalty,'elasticnet'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'saga'
,max_iter,100
,multi_class,'deprecated'


In [32]:
#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.85      0.91      0.88      5033
    Negative       0.90      0.83      0.87      4884

    accuracy                           0.87      9917
   macro avg       0.87      0.87      0.87      9917
weighted avg       0.87      0.87      0.87      9917

              precision    recall  f1-score   support

    Positive       0.92      0.88      0.90      5033
    Negative       0.88      0.92      0.90      4884

    accuracy                           0.90      9917
   macro avg       0.90      0.90      0.90      9917
weighted avg       0.90      0.90      0.90      9917



In [47]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(pd.DataFrame(cm_bow))
print()
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(pd.DataFrame(cm_tfidf))

      0     1
0  4064   820
1   445  4588

      0     1
0  4479   405
1   593  4440


In [37]:
mnb=MultinomialNB()

mnb_bow=mnb.fit(cv_test_reviews,train_sentiments)
print(mnb_bow)

mnb_tfidf=mnb.fit(tv_test_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


In [41]:
mnb_bow_predict=mnb.predict(cv_train_reviews)
print(mnb_bow_predict)
mnb_tfidf_predict=mnb.predict(tv_train_reviews)
print(mnb_tfidf_predict)

[1 1 1 ... 1 1 0]
[1 1 1 ... 1 1 0]


In [42]:
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.8754663708782898
mnb_tfidf_score : 0.8827266310376122


In [43]:
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)

mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.88      0.87      0.88      5033
    Negative       0.87      0.88      0.87      4884

    accuracy                           0.88      9917
   macro avg       0.88      0.88      0.88      9917
weighted avg       0.88      0.88      0.88      9917

              precision    recall  f1-score   support

    Positive       0.90      0.86      0.88      5033
    Negative       0.86      0.91      0.88      4884

    accuracy                           0.88      9917
   macro avg       0.88      0.88      0.88      9917
weighted avg       0.88      0.88      0.88      9917



In [44]:
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[4310  574]
 [ 661 4372]]
[[4426  458]
 [ 705 4328]]


In [72]:
import joblib
joblib.dump(best_model, "sentiment_reviewer_model.joblib")
joblib.dump(tv, "tfidf_vectorizer.joblib")

['tfidf_vectorizer.joblib']